In [50]:
import pandas as pd
from src import utils

#### Loading data

In [51]:
mutations = pd.read_csv('Data/CCLE_mutations.csv')
expression = pd.read_csv('Data/CCLE_expression.csv', index_col=0)
cnv = pd.read_csv('Data/CCLE_gene_cn.csv', index_col=0)

C:\Users\hp\AppData\Local\Temp\ipykernel_18088\293479094.py:1: DtypeWarning: Columns (3,19,22,27,28,29,30,31) have mixed types. Specify dtype option on import or set low_memory=False.
  mutations = pd.read_csv('Data/CCLE_mutations.csv')


In [52]:
prot_normalized = pd.read_csv('Data/protein_quant_current_normalized.csv', index_col=0)
sample_info = pd.read_csv('Data/sample_info.csv')

In [53]:
id_to_name_map = dict(zip(sample_info['DepMap_ID'], sample_info['CCLE_Name']))

#### Mutations processing

In [54]:
mutations

,Hugo_Symbol,Entrez_Gene_Id,NCBI_Build,Chromosome,Start_position,End_position,Strand,Variant_Classification,Variant_Type,Reference_Allele,...,isCOSMIChotspot,COSMIChsCnt,ExAC_AF,Variant_annotation,CGA_WES_AC,HC_AC,RD_AC,RNAseq_AC,SangerWES_AC,WGS_AC
0,VPS13D,55187,37,1,12359347,12359347,+,Nonsense_Mutation,SNP,C,...,False,0.0,NaN,damaging,34:213,NaN,NaN,NaN,NaN,NaN
1,AADACL4,343066,37,1,12726308,12726322,+,In_Frame_Del,DEL,CTGGCGTGACGCCAT,...,False,3.0,NaN,other non-conserving,57:141,NaN,NaN,NaN,NaN,NaN
2,IFNLR1,163702,37,1,24484172,24484172,+,Silent,SNP,G,...,False,0.0,NaN,silent,118:0,NaN,NaN,NaN,NaN,NaN
3,TMEM57,55219,37,1,25785018,25785019,+,Frame_Shift_Ins,INS,-,...,False,0.0,NaN,damaging,NaN,NaN,NaN,6:28,NaN,NaN
4,ZSCAN20,7579,37,1,33954141,33954141,+,Missense_Mutation,SNP,T,...,False,0.0,NaN,other non-conserving,28:62,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1293711,RBM41,55285,37,X,106310813,106310813,+,Missense_Mutation,SNP,C,...,False,0.0,NaN,other non-conserving,15:91,NaN,NaN,NaN,NaN,NaN
1293712,COL4A5,1287,37,X,107858220,107858220,+,Silent,SNP,A,...,False,0.0,NaN,silent,76:236,NaN,NaN,NaN,NaN,NaN
1293713,MBNL3,55796,37,X,131524971,131524971,+,Silent,SNP,C,...,False,0.0,NaN,silent,73:32,NaN,NaN,NaN,NaN,NaN
1293714,MAP7D3,79649,37,X,135314027,135314027,+,Silent,SNP,C,...,False,0.0,NaN,silent,78:175,NaN,NaN,NaN,NaN,NaN


In [55]:
mutations['mutation'] = 1


In [56]:
binary_matrix = mutations.pivot_table(
    index='DepMap_ID', 
    columns='Hugo_Symbol', 
    values='mutation', 
    fill_value=0
)


In [57]:
binary_matrix.head()

Hugo_Symbol,A1BG,A1CF,A2M,A2ML1,A3GALT2,A4GALT,A4GNT,AAAS,AACS,AADAC,...,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3,hsa-mir-1199
DepMap_ID,,,,,,,,,,,,,,,,,,,,,
ACH-000001,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
ACH-000002,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
ACH-000003,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
ACH-000004,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ACH-000005,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [58]:
binary_matrix.shape

(1749, 19543)

In [59]:
# Select relevant columns for encoding mutation types
mutations['count'] = 1
mutation_types = mutations.pivot_table(
    index='DepMap_ID', 
    columns=['Hugo_Symbol', 'Variant_Classification'], 
    values='count', 
    fill_value=0
)

# Flatten multi-level columns
mutation_types.columns = ['_'.join(col).strip() for col in mutation_types.columns.values]



In [60]:
# Display the encoded mutation types
mutation_types.head()

,A1BG_Frame_Shift_Del,A1BG_Frame_Shift_Ins,A1BG_In_Frame_Del,A1BG_Missense_Mutation,A1BG_Nonsense_Mutation,A1BG_Silent,A1BG_Splice_Site,A1CF_De_novo_Start_OutOfFrame,A1CF_Intron,A1CF_Missense_Mutation,...,ZZEF1_Splice_Site,ZZZ3_Frame_Shift_Del,ZZZ3_Missense_Mutation,ZZZ3_Nonsense_Mutation,ZZZ3_Silent,ZZZ3_Splice_Site,hsa-mir-1199_Frame_Shift_Del,hsa-mir-1199_Missense_Mutation,hsa-mir-1199_Silent,hsa-mir-1199_Splice_Site
DepMap_ID,,,,,,,,,,,,,,,,,,,,,
ACH-000001,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ACH-000002,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ACH-000003,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ACH-000004,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ACH-000005,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [61]:
mutation_types.shape

(1749, 95686)

In [62]:
mutation_features = pd.concat([binary_matrix, mutation_types], axis=1)

In [63]:
mutation_features.index

Index(['ACH-000001', 'ACH-000002', 'ACH-000003', 'ACH-000004', 'ACH-000005',
       'ACH-000006', 'ACH-000007', 'ACH-000008', 'ACH-000009', 'ACH-000010',
       ...
       'ACH-002462', 'ACH-002463', 'ACH-002464', 'ACH-002466', 'ACH-002467',
       'ACH-002475', 'ACH-002508', 'ACH-002509', 'ACH-002510', 'ACH-002511'],
      dtype='object', name='DepMap_ID', length=1749)

In [64]:
mutation_features = mutation_features.rename(index=id_to_name_map)
mutation_processed = utils.process(mutation_features)

Dimensions: (1749, 115229)


In [65]:
mutation_processed.index

Index(['127399_SOFT_TISSUE', '143B_BONE', '201T_LUNG', '21NT_BREAST',
       '22RV1_PROSTATE', '2313287_STOMACH', '253JBV_URINARY_TRACT',
       '253J_URINARY_TRACT', '42MGBA_CENTRAL_NERVOUS_SYSTEM', '451LU_SKIN',
       ...
       'YD15_SALIVARY_GLAND', 'YD38_UPPER_AERODIGESTIVE_TRACT',
       'YD8_UPPER_AERODIGESTIVE_TRACT', 'YH13_CENTRAL_NERVOUS_SYSTEM',
       'YKG1_CENTRAL_NERVOUS_SYSTEM', 'YMB1E_BREAST', 'YSCCC_BILIARY_TRACT',
       'YT_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE', 'ZR751_BREAST',
       'ZR7530_BREAST'],
      dtype='object', name='DepMap_ID', length=1749)

#### Copy number encoding

In [66]:
cnv

,A1BG (1),NAT2 (10),ADA (100),CDH2 (1000),AKT3 (10000),GAGE12F (100008586),RNA5-8SN5 (100008587),RNA18SN5 (100008588),RNA28SN5 (100008589),LINC02584 (100009613),...,HNRNPDL (9987),DMTF1 (9988),PPP4R1 (9989),CDH1 (999),SLC12A6 (9990),PTBP3 (9991),KCNE2 (9992),DGCR2 (9993),CASP8AP2 (9994),SCO2 (9997)
ACH-000001,1.179621,1.098639,1.199020,0.733573,1.009197,1.607905,1.177329,1.177329,1.177329,1.001387,...,0.987165,0.953195,1.385347,0.748888,0.930416,0.742241,1.398255,1.177329,0.995021,0.747212
ACH-000002,1.009801,0.981249,1.012577,1.315756,1.009073,0.611316,1.022060,1.022060,1.022060,0.996967,...,0.998968,1.004020,1.315756,1.015739,0.986230,1.005236,1.006528,1.022060,1.001574,1.022060
ACH-000003,1.022828,1.067509,1.524481,0.619630,1.001801,1.006063,0.950577,0.950577,0.950577,1.397002,...,0.923635,1.116181,0.994162,1.175473,0.876896,1.018163,0.985919,0.244061,0.969969,0.244061
ACH-000004,1.232225,1.061091,0.836336,1.054544,1.060779,1.018872,1.035168,1.035168,1.035168,1.071960,...,0.821476,0.833216,1.032713,0.824302,1.087132,0.820478,1.577572,1.098392,1.085809,0.822388
ACH-000005,1.151880,1.112505,0.816603,1.051343,1.084702,0.523496,0.844142,0.844142,0.844142,1.099404,...,0.867494,0.821698,0.843875,0.813705,1.119510,0.823270,1.726203,0.844142,1.098309,0.812665
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ACH-002510,1.054755,0.960984,0.961169,1.134161,1.506412,0.392116,1.160694,1.160694,1.160694,1.138698,...,0.713536,1.149565,1.134161,0.694136,0.992001,0.704184,1.141474,1.160694,1.141887,1.160694
ACH-002511,1.174349,0.693474,1.327255,0.885284,1.075771,1.077253,1.125935,1.125935,1.125935,1.336506,...,0.951820,0.681380,1.153535,0.929665,0.698785,0.929563,1.175898,1.125935,0.703827,1.121955
ACH-001421,0.998091,0.963633,0.979534,0.946760,1.258505,0.921985,0.968023,0.968023,0.968023,0.985983,...,0.971288,0.976895,0.946760,0.978355,0.977533,1.294315,1.293088,0.968023,0.958595,0.968023
ACH-001711,1.077586,1.249580,1.125984,1.065911,1.113815,0.686665,0.905190,0.905190,0.905190,1.127647,...,1.044843,1.089226,1.065911,0.855166,1.062906,0.910256,1.092677,0.905190,1.105859,0.905190


In [67]:
cnv = cnv.rename(index=id_to_name_map)
cnv_processed = utils.process(cnv)

Dimensions: (1745, 27639)


In [68]:
cnv_processed

,A1BG (1),NAT2 (10),ADA (100),CDH2 (1000),AKT3 (10000),GAGE12F (100008586),RNA5-8SN5 (100008587),RNA18SN5 (100008588),RNA28SN5 (100008589),LINC02584 (100009613),...,HNRNPDL (9987),DMTF1 (9988),PPP4R1 (9989),CDH1 (999),SLC12A6 (9990),PTBP3 (9991),KCNE2 (9992),DGCR2 (9993),CASP8AP2 (9994),SCO2 (9997)
127399_SOFT_TISSUE,1.046322,0.822284,1.112617,0.449009,1.076339,0.798868,1.256763,1.256763,1.256763,1.289088,...,0.822547,1.084064,0.810950,1.293828,0.800913,1.461852,1.301498,1.256763,1.082165,1.035862
1321N1_CENTRAL_NERVOUS_SYSTEM,0.924865,0.999100,1.570466,1.734802,1.132583,1.192943,0.087093,0.087093,0.087093,0.920093,...,0.901882,1.163587,1.308910,0.946048,0.936225,1.136013,0.953040,0.087093,0.955363,0.941295
143B_BONE,0.909567,0.928730,1.388889,0.892610,0.900258,0.394791,0.958561,0.958561,0.958561,1.409868,...,0.921911,0.972431,1.313006,1.086261,1.178011,0.896625,0.897403,0.958561,0.823279,0.865977
201T_LUNG,1.127234,0.665240,0.929903,0.934171,1.155175,0.701951,1.139892,1.139892,1.139892,0.784334,...,0.706207,1.131896,1.010535,1.055121,0.982482,1.078050,1.127850,1.139892,0.977381,1.021483
22RV1_PROSTATE,0.985060,1.282454,0.979420,0.968982,1.299975,0.600237,0.973633,0.973633,0.973633,0.981450,...,1.001410,1.301814,0.968982,0.984313,0.980324,0.975376,0.986454,0.973633,0.983048,0.969276
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YMB1_BREAST,0.942014,0.998251,1.160656,1.175641,1.886314,0.988843,1.351307,1.351307,1.351307,1.951554,...,0.977036,1.189458,1.154807,1.505822,1.170632,0.955411,0.764783,1.351307,0.957253,0.594458
YSCCC_BILIARY_TRACT,1.023091,0.741544,1.617077,0.727856,1.214903,1.236913,1.194533,1.194533,1.194533,1.015347,...,0.782801,0.786709,0.727856,1.210288,1.203196,0.917141,0.974839,1.194533,1.036409,1.194533
YT_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,0.991489,0.963016,1.323751,0.829142,1.156519,0.593294,1.010639,1.010639,1.010639,1.006279,...,0.987341,1.155971,1.173226,1.010486,0.848806,0.823994,0.995930,1.010639,1.009502,1.010639
ZR751_BREAST,1.121034,0.936824,1.355238,1.203580,1.501776,0.988743,0.390379,0.390379,0.390379,2.136030,...,0.987927,0.986493,1.203580,1.509959,1.076434,0.985081,0.740533,1.282646,0.830420,0.745992


#### Expression

In [69]:
expression = expression.rename(index=id_to_name_map)
expression_processed = utils.process(expression)

Dimensions: (1375, 19182)


In [70]:
expression_processed

,TSPAN6 (7105),TNMD (64102),DPM1 (8813),SCYL3 (57147),C1orf112 (55732),FGR (2268),CFH (3075),FUCA2 (2519),GCLC (2729),NFYA (4800),...,AC090517.4 (54816),AL160269.1 (11046),ABCF2-H2BE1 (114483834),FO393400.1 (149951),POLR2J3 (548644),H2BE1 (114483833),AL445238.1 (647264),GET1-SH3BGR (106865373),AC024598.1 (22891),AC113348.1 (102724657)
127399_SOFT_TISSUE,3.954196,0.0,6.816600,2.025029,3.931683,0.070389,0.070389,5.431957,3.810443,3.451541,...,0.176323,0.000000,2.823749,0.000000,4.375039,0.000000,0.000000,0.443607,0.000000,0.056584
143B_BONE,5.421897,0.0,7.310340,1.773996,3.789729,0.014355,0.695994,5.385431,4.619413,3.436961,...,0.084064,0.000000,0.422233,0.042644,2.592158,0.000000,0.000000,0.000000,0.000000,0.000000
22RV1_PROSTATE,2.643856,0.0,6.219556,3.418190,4.659925,0.014355,0.111031,5.846243,7.057883,5.288359,...,0.042644,0.056584,1.124328,0.028569,5.069960,0.000000,0.000000,1.803227,0.014355,0.000000
2313287_STOMACH,2.985500,0.0,6.778734,4.130931,3.778209,0.000000,0.298658,7.433794,6.689299,4.291309,...,0.367371,0.124328,2.192194,0.000000,4.358959,0.000000,0.042644,0.097611,0.000000,0.000000
253JBV_URINARY_TRACT,4.553361,0.0,5.830103,1.867896,3.581351,0.000000,0.084064,5.014355,4.447579,3.919340,...,0.084064,0.298658,3.241840,0.150560,5.899901,0.111031,0.000000,0.356144,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YH13_CENTRAL_NERVOUS_SYSTEM,4.863938,0.0,5.977967,2.553361,3.772941,0.275007,7.178814,6.867279,3.959770,4.715893,...,0.000000,0.000000,3.748461,0.000000,6.263973,0.000000,0.000000,0.111031,0.150560,0.000000
YKG1_CENTRAL_NERVOUS_SYSTEM,5.914086,0.0,6.749668,2.809414,4.175525,0.176323,5.859224,6.535275,4.598127,5.063071,...,0.097611,0.495695,3.379898,0.238787,6.094869,0.000000,0.000000,1.028569,0.028569,0.000000
YSCCC_BILIARY_TRACT,5.975676,0.0,7.763943,3.124328,3.297191,0.056584,6.578184,6.383359,7.683065,4.161081,...,0.201634,0.389567,1.811471,0.028569,4.173127,0.000000,0.000000,0.910733,0.000000,0.042644
ZR751_BREAST,3.158660,0.0,7.092229,4.063503,4.101818,0.056584,1.130931,4.381283,4.049631,3.714795,...,0.000000,0.000000,0.000000,0.189034,5.883376,0.000000,0.000000,1.286881,0.000000,0.000000


#### Proteomics

In [71]:
prot_normalized.set_index('Gene_Symbol', inplace=True)
proteomics = prot_normalized.loc[:,  prot_normalized.columns.str.contains('_TenPx')]
proteomics.drop(columns=['SW948_LARGE_INTESTINE_TenPx11', 'CAL120_BREAST_TenPx02', 'HCT15_LARGE_INTESTINE_TenPx30'], 
                    inplace=True)
proteomics = proteomics.rename(columns = lambda x : str(x).split('_TenPx')[0])
proteomics_processed = utils.process(proteomics)

Dimensions: (5108, 375)


C:\Users\hp\AppData\Local\Temp\ipykernel_18088\2743761720.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  proteomics.drop(columns=['SW948_LARGE_INTESTINE_TenPx11', 'CAL120_BREAST_TenPx02', 'HCT15_LARGE_INTESTINE_TenPx30'],


In [72]:
proteomics_processed = proteomics_processed.transpose()

In [73]:
proteomics_processed.index

Index(['MDAMB468_BREAST', 'SH4_SKIN', 'AU565_BREAST', 'KMRC1_KIDNEY',
       'CAL51_BREAST', 'RPMI7951_SKIN', 'RERFLCMS_LUNG', 'IGR37_SKIN',
       'VMRCRCW_KIDNEY', 'HEP3B217_LIVER',
       ...
       'NCIH2030_LUNG', '22RV1_PROSTATE', 'A172_CENTRAL_NERVOUS_SYSTEM',
       'BT20_BREAST', 'CALU6_LUNG', 'FADU_UPPER_AERODIGESTIVE_TRACT',
       'KP4_PANCREAS', 'MONOMAC6_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE',
       'OVCAR8_OVARY', 'THP1_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE'],
      dtype='object', length=375)

#### Matching Samples

In [74]:
common_cell_lines = set(cnv.index) & set(mutation_features.index) & set(expression.index) & set(proteomics_processed.index)
len(common_cell_lines)


368

In [75]:
common_cell_lines = list(common_cell_lines)
cnv_data = cnv_processed.loc[common_cell_lines]
mutation_data = mutation_processed.loc[common_cell_lines]
mrna_data = expression_processed.loc[common_cell_lines]
protein_data = proteomics_processed.loc[common_cell_lines]
combined_features = pd.concat([mrna_data, cnv_data, mutation_data], axis=1)
combined_features

,TSPAN6 (7105),TNMD (64102),DPM1 (8813),SCYL3 (57147),C1orf112 (55732),FGR (2268),CFH (3075),FUCA2 (2519),GCLC (2729),NFYA (4800),...,ZZEF1_Splice_Site,ZZZ3_Frame_Shift_Del,ZZZ3_Missense_Mutation,ZZZ3_Nonsense_Mutation,ZZZ3_Silent,ZZZ3_Splice_Site,hsa-mir-1199_Frame_Shift_Del,hsa-mir-1199_Missense_Mutation,hsa-mir-1199_Silent,hsa-mir-1199_Splice_Site
MIAPACA2_PANCREAS,0.097611,0.000000,6.579391,1.769772,3.681449,0.028569,0.176323,6.074891,3.443607,4.140779,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
OVSAHO_OVARY,5.533563,0.000000,7.121948,2.295723,4.121015,0.014355,2.726831,5.361768,3.224966,4.782933,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SBC5_LUNG,3.783457,0.070389,5.909053,2.503349,3.282440,0.014355,0.124328,0.286881,3.871844,4.177918,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ISHIKAWAHERAKLIO02ER_ENDOMETRIUM,5.611468,0.000000,5.754888,2.046142,3.527321,0.084064,0.871844,6.628919,4.710944,4.339137,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
JIMT1_BREAST,5.163096,0.000000,7.580447,2.531069,4.802710,0.150560,0.333424,5.845490,4.378512,4.021480,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HCC1937_BREAST,5.584963,0.000000,7.069853,2.750607,4.658211,0.150560,0.070389,5.778209,4.366672,3.930737,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CAL27_UPPER_AERODIGESTIVE_TRACT,4.361066,0.000000,6.446256,2.526069,4.106013,0.250962,4.877253,5.753284,5.247548,3.914565,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
HCC1143_BREAST,5.890690,0.000000,6.181500,3.177918,4.667892,0.042644,0.422233,6.553975,5.146900,5.345893,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SJSA1_BONE,3.990047,0.000000,6.360891,2.550901,4.383359,0.056584,5.987321,5.268285,5.458776,4.622930,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
